In [1]:
import re
import os, json
from dotenv import load_dotenv
from textwrap import dedent
from pprint import pprint
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import requests
response = requests.get("http://127.0.0.1:11434")
print(response.text)
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

Ollama is running


In [2]:
# 기본 유틸
import os
import json
import re
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

# 문서 로딩 & 전처리
from langchain.document_loaders import Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 벡터화 (임베딩) / 검색
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# LLM 모델
#from langchain.chat_models import ChatOpenAI
# (만약 Ollama 쓰면)
from langchain_ollama import ChatOllama

# 체인 및 에이전트
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.agents import initialize_agent, Tool

# 외부 API 호출 등 필요 시
import requests


In [3]:
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
llm_model = ChatOllama(model="qwen3:1.7b",temperature=0.1)

In [5]:
# 2. 문서 로딩 및 분할
resume_path = Path("./data/tutorial-korean.pdf")  # sample_resume.pdf 또는 sample_resume.docx 경로 지정

# 로더 선택
if resume_path.suffix.lower() == ".pdf":
    loader = PyPDFLoader(str(resume_path))
else:
    loader = Docx2txtLoader(str(resume_path))

# 로딩 및 분할
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
print(f"분할된 청크 개수: {len(chunks)}")

# 3. 임베딩 및 벡터스토어 구축
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore = FAISS.from_documents(chunks, embeddings)

# 4. RetrievalQA 체인 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "아래는 지원자 이력서에서 추출된 정보입니다:\n"
        "{context}\n\n"
        "위 정보를 참고해, '{question}'에 대해 답변해 주세요."
    )
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt}
)
# 5. 예시 질의
question = "백엔드 개발자 포지션에 지원할 때 강조하면 좋을 기술은 무엇일까?"
answer = qa_chain.run(question)
print("▶️ AI 답변:\n", answer)


분할된 청크 개수: 66
▶️ AI 답변:
 지원자는 디버깅에 대한 기본적인 지식과 BlueJ를 사용한 디버깅 경험이 있음을 보여주고 있습니다. 이는 백엔드 개발자로서 버그를 찾고 해결하는 능력이 중요하기 때문에 강조할 수 있는 기술입니다. 또한, 중단점 설정, 단계별 코드 실행, 변수 검사 등의 디버깅 기능을 소개하고 있으므로 이러한 기능을 통해 코드를 분석하고 수정하는 능력을 갖추고 있다는 것을 강조할 수 있습니다. 또한, 독립형 어플리케이션 및 애플릿을 만들고 관련 문서를 생성하는 경험도 언급되어 있으므로 이러한 능력 또한 백엔드 개발자로서 유용하게 활용할 수 있을 것입니다.


NameError: name 'StateGraph' is not defined